# How to:

Download images from ISIC using the API in three steps.

### First Step:

Import cascid module's automated download functions.

In [ ]:
# Import isic downloading tools
from cascid.datasets.isic import database, fetcher

### Second Step:

Ask fetcher to gather the metadata for however many images you want from each class. The 'fetch_from_isic' function returns a list of objects, which contain the metadata. This list is promptly passed on to the 'save_metadata' function, which interprets these objects, and saves a dataframe to a .csv file. This operation is repeatable, and will never overwrite previously collected data. As such, you have the option of running the cell below multiple times, without fear of loose metadata files, or losing access to already downloaded images. 

In [ ]:
images = fetcher.fetch_from_isic(
    n_samples=6000, 
    diagnosis_list=[
        "melanoma",
        "nevus",
        '"basal cell carcinoma"',
        '"seborrheic keratosis"',
        '"actinic keratosis"',
        '"squamous cell carcinoma"'
    ]
)
fetcher.save_metadata(image_list=images)

The 'database.get_df' function, reads from this csv automatically.

In [ ]:
df = database.get_df()
df

### Third Step:

So far, we have only gathered the metadata for the images, but not downloaded any of the images yet.
Supply the read dataframe to the 'update_all_files' function. This function can detect missing images in the storage directory, and download them automatically based on the metadata available. Since the download is very slow on the network IO side, this is done using multiple threads, for a big performance boost. It can still be rather slow, as some images are very large. This can also produce some timeout errors if your network is particularly slow, but simply rerunning the cell will resume progress from where it left off, and continue downloading.  

In [ ]:
database.update_all_files(df)

And that's it! Now you have as many images as you want from the ISIC dataset. You may notice, you asked for 'x' images of each diagnosis, but only got 'y'. ISIC is not endless, and some diagnosis have more images than others. If one diagnosis has no more images, no more images will be downloaded. The program will still attempt to reach the goal for each diagnosis separately though, so if you ask for 5000, and there are 5000, you should get pretty close to 5000 (some images lack basic metadata, such as age or gender, and as such are not downloaded at all, bringing the total down by a few images on occasion).